In [1]:
# Used to read the Parquet data
import pyarrow.parquet as parquet
# Used to train the baseline model
from sklearn.linear_model import LogisticRegression

# Where the downloaded data are
input_path = './'
# Where to store results
output_path = './'

In [2]:
# Read a single day to train model on as Pandas dataframe
data = parquet.read_table(input_path + '/collabTrain/date=2018-02-07').to_pandas()
data.head(10)

/Users/dmitriybugaichenko/anaconda/envs/python37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


,instanceId_userId,instanceId_objectType,instanceId_objectId,audit_pos,audit_clientType,audit_timestamp,audit_timePassed,audit_experiment,audit_resourceType,metadata_ownerId,...,auditweights_userOwner_USER_INTERNAL_UNLIKE,auditweights_userOwner_USER_PRESENT_SEND,auditweights_userOwner_USER_PROFILE_VIEW,auditweights_userOwner_USER_SEND_MESSAGE,auditweights_userOwner_USER_STATUS_COMMENT_CREATE,auditweights_userOwner_VIDEO,auditweights_userOwner_VOTE_POLL,auditweights_x_ActorsRelations,auditweights_likersSvd_spark_hyper,auditweights_source_PROMO
0,189,Post,19851617,10,API,1517999095791,784813,XPRM-5386_q4x2,8,35736,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,384,Post,11221208,7,WEB,1517961558224,2972959,XPRM-5386_q2x2,8,25338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,384,Post,22510814,11,WEB,1517961558224,2972959,XPRM-5386_q2x2,8,14962,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,384,Post,12870307,17,WEB,1517961558224,2972959,XPRM-5386_q2x2,8,46044,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,384,Post,12932713,24,WEB,1518033144740,277687,XPRM-5386_q2x2,8,18709,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1017,Post,18318017,86,API,1517996513416,42757268,XPRM-5386_q2x3,8,17234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1371,Post,9899727,22,WEB,1517993615130,134506,XPRM-5386_q2x0,8,65392,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1464,Post,30609611,8,MOB,1518024581765,40050084,XPRM-5386_q2x3,8,37132,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1917,Post,24769843,1,MOB,1518019007598,6813431,XPRM-5386_G2,8,27607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2076,Post,16648806,12,WEB,1518032656355,4530576,XPRM-5386_q4x1,8,36405,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN


In [3]:
# Construct the label (liked objects)
y = data['feedback'].apply(lambda x: 1.0 if("Liked" in x) else 0.0).values

# Extract the most interesting features
X = data[[
        'auditweights_svd_prelaunch', 
        'auditweights_ctr_high', 
        'auditweights_ctr_gender', 
        'auditweights_friendLikes']].fillna(0.0).values

In [4]:
# Fit the model and check the weights
model = LogisticRegression(random_state=0, solver='lbfgs').fit(X, y)
model.coef_

array([[ 4.6212144 ,  2.69850023, -0.19539506,  0.12107017]])

In [5]:
# Read the test data
test = parquet.read_table(input_path + '/collabTest').to_pandas()
test.head(10)

/Users/dmitriybugaichenko/anaconda/envs/python37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


,instanceId_userId,instanceId_objectType,instanceId_objectId,audit_pos,audit_clientType,audit_timestamp,audit_timePassed,audit_experiment,audit_resourceType,metadata_ownerId,...,auditweights_userOwner_USER_INTERNAL_UNLIKE,auditweights_userOwner_USER_PRESENT_SEND,auditweights_userOwner_USER_PROFILE_VIEW,auditweights_userOwner_USER_SEND_MESSAGE,auditweights_userOwner_USER_STATUS_COMMENT_CREATE,auditweights_userOwner_VIDEO,auditweights_userOwner_VOTE_POLL,auditweights_x_ActorsRelations,auditweights_likersSvd_spark_hyper,auditweights_source_PROMO
0,681,Post,35119221,7,API,1521813396697,360522,XPRM-5386_G10,8,14952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.927496,NaN
1,681,Post,35151632,32,API,1521904815253,3314178,XPRM-5386_G10,8,9182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,681,Post,20314212,9,API,1522394611865,803588,XPRM-5386_G10,8,16395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,512.0,NaN,NaN
3,1428,Post,27968107,3,API,1522205864350,27994833,XPRM-5386_G4,8,15993,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,512.0,NaN,NaN
4,1428,Post,24530710,2,API,1522515569257,753104,XPRM-5386_G4,8,53860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,NaN,NaN
5,2064,Post,39107414,2,API,1521991610627,2558537,XPRM-5386_G9,8,61558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2064,Post,36600601,2,API,1521993892399,708736,XPRM-5386_G9,8,19761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2388,Post,23246730,1,MOB,1522304987799,75927584,XPRM-5386_G5,8,68570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2388,Post,30828036,2,MOB,1522355681967,203214,XPRM-5386_G5,8,78974,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2706,Post,38122236,7,MOB,1521923473536,23161418,XPRM-5386_G9,8,81707,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Compute inverted predictions (to sort by later)
test["predictions"] = -model.predict_proba(test[[
        'auditweights_svd_prelaunch', 
        'auditweights_ctr_high', 
        'auditweights_ctr_gender', 
        'auditweights_friendLikes']].fillna(0.0).values)[:, 1]

In [7]:
# Peek only needed columns and sort
result = test[["instanceId_userId", "instanceId_objectId", "predictions"]].sort_values(
    by=['instanceId_userId', 'predictions'])
result.head(10)

,instanceId_userId,instanceId_objectId,predictions
337303,39,35321415,-0.205321
337300,39,18936317,-0.060567
337301,39,38436621,-0.029520
337302,39,38345738,-0.008043
1010995,78,14594128,-0.249085
1010994,78,38727626,-0.191624
1010993,78,549335,-0.104174
676506,135,17551814,-0.362909
676504,135,17738426,-0.344983
676513,135,18057011,-0.292308


In [8]:
# Collect predictions for each user
submit = result.groupby("instanceId_userId")['instanceId_objectId'].apply(list)
submit.head(10)

instanceId_userId
39               [35321415, 18936317, 38436621, 38345738]
78                           [14594128, 38727626, 549335]
135     [17551814, 17738426, 18057011, 34080312, 38102...
180                                  [35804715, 12537921]
231                [30371401, 34978207, 975302, 30978017]
279                                   [12179433, 9471943]
681                        [20314212, 35151632, 35119221]
873                                  [18419723, 15483310]
1071    [25539638, 23720423, 36006622, 25050009, 35834...
1113                       [35894718, 29841332, 15781817]
Name: instanceId_objectId, dtype: object

In [9]:
# Persist the first submit
submit.to_csv(output_path + "/collabSubmit.csv.gz", header = False, compression='gzip')